In [ ]:
!pip3 install tensorboardx
    
import datetime, os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import pickle
import cv2
from io import BytesIO
from PIL import Image
import zipfile
import gc
import math, os
from random import shuffle
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/datasets/data_ad_anogan.zip ./
!7z x data_ad_anogan.zip

Mounted at /content/drive

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 680158946 bytes (649 MiB)

Extracting archive: data_ad_anogan.zip
--
Path = data_ad_anogan.zip
Type = zip
Physical Size = 680158946

  0%      7% 103 - data/capsule/test/scratch/022.png                                             12% 167         19% 240 - data/capsule/train/good/144.png                                           25% 304 - data/pill/train/good/033.png                                        32% 372 - data/pi

In [ ]:
class PillsDataset(Dataset):
    def __init__(self, type='train', _transforms=None):
        assert type in ['test', 'train', 'ground_truth']
        self.type = type
        self.transforms = _transforms
        self.samples = []
        self.toPIL = torchvision.transforms.ToPILImage()
        self.toTensor = torchvision.transforms.ToTensor()
        for root,d_names,f_names in os.walk("/content/data/capsule/"+type+"/good"):
          for _file in f_names:
            filename = os.path.join(root, _file)
            self.samples.append(filename)
        print(f"dataset_len: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        name = self.samples[index]
        sample = Image.open(name).convert('RGB')
        if self.transforms:
            sample = self.transforms(sample)
        return sample

BS = 64
train_transform = transforms.Compose(
        [
            transforms.Resize(256),
         
            #transforms.ColorJitter(saturation=0.3, contrast=0.3, brightness=0.3),
            transforms.ToTensor(),
            #transforms.Normalize((123.68 / 255, 116.779 / 255, 103.939/ 255), (58.393 / 255, 57.12 / 255, 57.375 / 255)),
            transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
        ]
    )
trainset = PillsDataset('train', _transforms=train_transform)
train_loader = DataLoader(trainset, batch_size=BS, shuffle=True, drop_last=True)

def prin_batch():
    TRAIN=1

    
    epochs = 0
    epoch = 0
    for images in train_loader:
      epoch+=1
      #trainset.get_label(label_indices)
      break
    
    img = images.data 
    img = img - img.min()
    img = img / img.max()
    img = make_grid(img, nrow=2, padding=1).numpy()
    img = np.rollaxis(img, 0, 3)

    plt.figure(figsize=(100, 100)), plt.imshow(img), plt.show()
    if False:
        plt.figure(figsize=(14, 5)), plt.imshow(img[:,:,0]), plt.show()
        plt.figure(figsize=(14, 5)), plt.imshow(img[:,:,1]), plt.show()
        plt.figure(figsize=(14, 5)), plt.imshow(img[:,:,2]), plt.show()

prin_batch()

In [ ]:
def MyConvTranspose(in_channel, out_channel):
  conv = nn.ConvTranspose2d(in_channel, out_channel, (2, 2), (2, 2), (0, 0), bias=False)
  nn.init.normal_(conv.weight.data, 0, 0.02)

  bn = nn.BatchNorm2d(out_channel, affine=True, track_running_stats=False)
  #torch.nn.init.zeros_(bn.weight)
  nn.init.normal_(bn.weight.data, 1.0, 0.02)
  nn.init.constant_(bn.bias.data, 0)

  seq = nn.Sequential(
    conv,
    bn,
    nn.ReLU()
  )
  return seq


class DC_generator(nn.Module):
  def __init__(self, latent_vector_size=100):
    super(DC_generator, self).__init__()
    self.latent_to_tensor = nn.Linear(latent_vector_size, 4*4*1024, bias=False)
    torch.nn.init.xavier_normal_(self.latent_to_tensor.weight)

    lastconv = nn.ConvTranspose2d(32, 3, (2, 2), (2, 2), (0, 0))
    nn.init.normal_(lastconv.weight.data, 0, 0.02)

    last_bn = nn.BatchNorm2d(3, affine=True, track_running_stats=False)
    nn.init.normal_(last_bn.weight.data, 1.0, 0.02)
    nn.init.constant_(last_bn.bias.data, 0)

    fbn = nn.BatchNorm2d(1024, affine=True, track_running_stats=False)
    
    nn.init.normal_(fbn.weight.data, 1.0, 0.02)
    nn.init.constant_(fbn.bias.data, 0)
    
    layers = [
                fbn,
                nn.ReLU(),
                MyConvTranspose(1024, 512),
                MyConvTranspose(512, 256),
                MyConvTranspose(256, 128),
                MyConvTranspose(128, 64),
                MyConvTranspose(64, 32),
                lastconv,  
                last_bn,
                nn.Tanh()        
             ]
    self.layers = nn.Sequential(
            *layers
        )
    
  def forward(self, x):
    x = self.latent_to_tensor(x)
    x = x.view(x.size(0), 1024, 4, 4)
    return self.layers(x)



def MyConv(in_channel, out_channel):
  conv = nn.Conv2d(in_channel, out_channel, (2, 2), (2, 2), (0, 0), bias=False)
  nn.init.normal_(conv.weight.data, 0, 0.02)

  bn = nn.BatchNorm2d(out_channel, affine=True, track_running_stats=False)
  nn.init.normal_(bn.weight.data, 1.0, 0.02)
  nn.init.constant_(bn.bias.data, 0)

  conv = nn.Sequential(
    conv,
    bn,
    nn.ReLU()
  )
  return conv

class DC_discriminator(nn.Module):
  def __init__(self):
    super(DC_discriminator, self).__init__()
    layers = [
                MyConv(3,32),
                MyConv(32,64),
                MyConv(64,128),
                MyConv(128,256),
                MyConv(256,512), 
                MyConv(512,1024),        
             ]
    self.layers = nn.Sequential(
            *layers
        )
    self.linear = nn.Linear(4*4*1024, 1, bias=False)
    torch.nn.init.xavier_normal_(self.linear.weight)
    
    
  def forward(self, x):
    #x = self.latent_to_tensor(x)
    #x = x.view(x.size(0), 1024, 4, 4)
    x = self.layers(x)
    x = x.view(x.size(0), -1)
    x = self.linear(x)
    return torch.sigmoid(x)

    
_input = torch.rand((2,100))
gen = DC_generator(100)
descr = DC_discriminator()
out = gen(_input)
print(out.size())
print(descr(out).size())

torch.Size([2, 3, 256, 256])
torch.Size([2, 1])


In [ ]:
G_LR = 0.0002
D_LR = 0.0002
CUDA = torch.cuda.is_available()
LATENT_DIM = 100
EPOCHS_COUNT = 1000
 
def denormalize(x):
  std = torch.tensor((58.393 / 255, 57.12 / 255, 57.375 / 255)).view(-1,1,1)
  mean = torch.tensor((123.68 / 255, 116.779 / 255, 103.939 / 255)).view(-1,1,1)
  if CUDA:
    std = std.cuda()
    mean= mean.cuda()
  out = x.mul_(std).add_(mean)
  out[out>1] = 1
  out[out<0] = 0
  return out
 
 
def save_images(batch, epoch):
  path_to, k = "generated_images/", 0
  if not os.path.exists(path_to):
      os.makedirs(path_to)
  batch = denormalize(batch)
  for image in batch:
      torchvision.utils.save_image(image, f"{path_to}{epoch}.jpg",
                                  normalize=False)
      break
!rm -rf /content/generated_images
def train_model():
 
  generator = DC_generator(LATENT_DIM)
  discriminator = DC_discriminator()
  if CUDA:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
  
  optimizer_G = torch.optim.Adam(generator.parameters(), lr=G_LR, betas=(0.5, 0.999),)
  optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=D_LR, betas=(0.5, 0.999),)
 
  CEloss = nn.BCELoss()
 
  ones = torch.ones((BS,1))
  zeros = torch.zeros((BS,1))
 
  if CUDA:
    ones = ones.cuda()
    zeros = zeros.cuda()
 
  for epoch in range(EPOCHS_COUNT):
    print(f"Epoch {epoch}/{EPOCHS_COUNT}")
    iter_count = len(train_loader)
    g_losses = []
    real_acc = []
    fake_acc = []
    d_losses = []
    real_losses = []
    fake_losses = []

    g_images = None
    for i, images in enumerate(train_loader):
      
      z = torch.rand((images.size(0), LATENT_DIM))
      if CUDA:
        images = images.cuda()
        z = z.cuda()
 
      n_critic = 1
      clip_param = 0 #0.01
      if 1: #i % n_critic == 0:
        optimizer_G.zero_grad()
        g_images = generator(z)
        g_loss = CEloss(discriminator(g_images), ones)
        #g_loss = -torch.mean(discriminator(g_images))
        g_loss.backward()
        optimizer_G.step()

 
      optimizer_D.zero_grad()

      d_fake_out = discriminator(g_images.detach())
      d_fake_loss = CEloss(d_fake_out, zeros)
      #d_fake_loss = torch.mean(d_fake_out)
      d_fake_loss.backward()

      optimizer_D.step()
      optimizer_D.zero_grad()

      d_real_out = discriminator(images)
      d_real_loss = CEloss(d_real_out, ones)
      #d_real_loss = -torch.mean(d_real_out)

      d_loss = (d_fake_loss + 0.1*d_real_loss)
      d_real_loss.backward()
      
      #d_loss.backward()
 
      optimizer_D.step()
 
      
      '''
      #wesserstein clipping
      if clip_param:
        for p in discriminator.parameters():
              p.data.clamp_(-clip_param, clip_param)
      '''

      g_losses.append(g_loss.item())
      d_losses.append(d_loss.item())
 
      real_acc.append(torch.mean(d_real_out).item())
      fake_acc.append(torch.mean(d_fake_out).item())
      real_losses.append(d_real_loss.item())
      fake_losses.append(d_fake_loss.item())
      print(f"\r\titeration {i+1}/{iter_count}: real_out: {real_acc[-1]}, fake_out: {fake_acc[-1]} g_loss: {g_losses[-1]}, real_loss: {real_losses[-1]}, fake_loss: {fake_losses[-1]}")
    avg = lambda x: sum(x)/len(x)
    save_images(g_images, epoch)
    print(f"  Epoch {epoch} result avg: real_out: {avg(real_acc)}, fake_out: {avg(fake_acc)} g_loss: {avg(g_losses)}, real_loss: {avg(real_losses)}, fake_loss: {avg(fake_losses)}")
 
 
train_model()